# Useful documentation links

- https://python.langchain.com/docs/get_started/introduction.html
- https://python.langchain.com/docs/use_cases/question_answering/
- https://github.com/langchain-ai/langchain
- https://github.com/langchain-ai/chat-langchain

# Loading in Secret key

In [ ]:
from langchain.llms import OpenAI
import os

# key using env
os.environ['openaiapikey']='sk-keyyyyy'
openai.api_key = os.getenv('openaiapikey')

#key using txt
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
api_key = f.read()

llm = ChatOpenAI(openai_api_key=api_key)

# Chat messages
- SystemMessage: the persona
- HumanMessage: a msg from the user pov
- AIMessage: the response to the human message

In [ ]:

from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

result = chat([SystemMessage(content='You are a very rude teenager'),
               HumanMessage(content='Can you tell me a fact about Earth?')]
               temperature = 0.1,  #lower = more focused and deterministic output, 0 to 2
               presence_penalty = 1, #penalise tokens that appear many times
               max_tokens = 100)

result.content #outputs an AI message

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (ChatPromptTemplate, PromptTemplate, 
    SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate)


def travel_idea(interest,budget):
    '''
    INPUTS:
        interest: A str interest or hobby (e.g. fishing)
        budget: A str budget (e.g. $10,000)
    '''
    # PART ONE: SYSTEM
    system_template="You are an AI Travel Agent that helps people plan trips about {interest} on a budget of {budget}"
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    
    # PART TWO: HUMAN REQUEST
    human_template="{travel_help_request}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    
    # PART THREE: COMPILE TO CHAT
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                    human_message_prompt])
    
    # PART FOUR: INSERT VARIABLES
    request = chat_prompt.format_prompt(interest=interest, \
                                        budget=budget, \
                                        travel_help_request="Please give me an example travel itinerary").to_messages()
    
    # PART FIVE: CHAT REQUEST
    chat = ChatOpenAI(openai_api_key=api_key, \
                      temperature=0, \
                      presence_penalty=0.5, \
                      max_tokens=100)
    result = chat(request)
    
    return result.content

#use input functions
activity_input = input("Enter an activity for your travel idea: ")
budget_input = input("Enter your budget for the trip: ")

print(activity_input)
print(budget_input)
print(travel_idea(activity_input,budget_input))

# Caching

In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.cache import InMemoryCache

langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a fact about Mars")

# You will notice this reply is instant!
# does not ping the api! 
llm.predict('Tell me a fact about Mars')

# Saving prompt ro Json

In [ ]:
from langchain import PromptTemplate

template = "Question: {question}\n\nAnswer: Let's think step by step."
prompt = PromptTemplate(template=template, input_variables=["question"])
prompt.save("prompt.json")

In [ ]:
# All prompts are loaded through the `load_prompt` function.
from langchain.prompts import load_prompt

loaded_prompt = load_prompt('prompt.json')
loaded_prompt

# Parsing output
date parsing example

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

model = ChatOpenAI(openai_api_key=api_key)

output_parser = DatetimeOutputParser()
print(output_parser.get_format_instructions())

In [ ]:
# system prompt to compliment output parser
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

result = model(request,temperature=0)
result.content

In [ ]:
# fix parsing issue: aut-fix parser
#consider system prompt template in combination

from langchain.output_parsers import OutputFixingParser

# output_parser = DatetimeOutputParser()
misformatted = result.content

new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)
new_parser.parse(misformatted)

# Few shot prompt: learning from example

In [ ]:
#importing
chat = ChatOpenAI(openai_api_key=api_key, temperature=0)

#example of a legal text
legal_text = "The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."
example_input_one = HumanMessagePromptTemplate.from_template(legal_text)

#example of the simplified (plain) text
plain_text = "The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law."
example_output_one = AIMessagePromptTemplate.from_template(plain_text)


#set up human prompt template
human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, 
     example_input_one, 
     example_output_one, 
     human_message_prompt])

#example input
some_example_text = "The grantor, being the fee simple owner of the real property herein described, conveys and warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, and any applicable covenants, conditions, and restrictions affecting the property, in consideration of the sum of [purchase price] paid by the grantee."
request = chat_prompt.format_prompt(legal_text=some_example_text).to_messages()
result = chat(request)
print(result.content)

# Connect to document and retrieve relavent results

In [ ]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

import os
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
os.environ['OPENAI_API_KEY'] = f.read()


def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!
    
    Follow the steps below to fill out this function:
    '''
    # PART ONE:
    # LOAD "some_data/US_Constitution in a Document object
    loader = TextLoader("some_data/US_Constitution.txt")
    documents = loader.load()
    
    # PART TWO
    # Split the document into chunks (you choose how and what size)
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
    docs = text_splitter.split_documents(documents)
    
    # PART THREE
    # EMBED THE Documents (now in chunks) to a persisted ChromaDB
    # embedding_function = OpenAIEmbeddings()
    embedding_function = OpenAIEmbeddings()
    db = Chroma.from_documents(docs, embedding_function, persist_directory='./US_Constitution')
    db.persist()

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.

    # results = db.similarity_search("What is the 13th Amendment?")
    # print(results[0].page_content) # NEED TO COMPRESS THESE RESULTS!
    llm = ChatOpenAI(temperature=0)
    compressor = LLMChainExtractor.from_llm(llm)

    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, 
                                                           base_retriever=db.as_retriever())

    compressed_docs = compression_retriever.get_relevant_documents(question)

    return compressed_docs[0].page_content

print(us_constitution_helper("What is the 13th Amendment?"))

# Connect to SQL
- https://python.langchain.com/docs/use_cases/tabular/
- https://python.langchain.com/docs/use_cases/tabular/sqlite

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# For data-sensitive projects, you can specify return_direct=True in the SQLDatabaseChain initialization to directly return the output of the SQL query without any additional formatting.
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")
llm = OpenAI(temperature=0, verbose=True)


db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
db_chain.run("How many employees are there?")

Sometimes the Language Model generates invalid SQL 
with small mistakes that can be 
self-corrected using the same technique 
used by the SQL Database Agent 
to try and fix the SQL using the LLM. 
You can simply specify this option when creating the chain

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)
db_chain.run("How many albums by Aerosmith?")

In [ ]:
#limit query to number of rows
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True, top_k=3)

# Sequential chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

llm = ChatOpenAI()


template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm,
                     prompt=prompt1,
                     output_key="review_summary")


template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=llm,
                     prompt=prompt2,
                     output_key="weaknesses")


template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=llm,
                     prompt=prompt3,
                     output_key="final_plan")


seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['review'],
                            output_variables=['review_summary','weaknesses','final_plan'],
                            verbose=True)

In [ ]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.
One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.
Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.
Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.
Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.
Weaknesses
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.
Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.
Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [ ]:
results = seq_chain(employee_review)
results #dictionary

In [ ]:
print(results['final_plan']) #index into dictionary

# Router chain
use case example: route the prompt to most appropriate LLM chain. 
eg. internal employees vs external clients

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()

In [ ]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge. Here is the question\n{input}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

In [ ]:
#route prompt info
#list of dict of name, description, template

prompt_infos = [
    {'name':'advanced physics',
    'description': 'Answers advanced physics questions',
     'prompt_template':expert_template},\
    \
    {'name':'beginner physics',
    'description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},]

In [ ]:
#dict that refer to llm
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)

    #add to dict
    destination_chains[name] = chain

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm = llm, 
                        prompt = default_prompt)

In [ ]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# print(MULTI_PROMPT_ROUTER_TEMPLATE)
# print to see more info and hints

# li comprehension for route destination
# for every prompt info, grab the key name: value description 
# output is li of key:value
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

#join them to create a single string
destinations_str = "\n".join(destinations)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

# now you should see an output similar to print(MULTI_PROMPT_ROUTER_TEMPLATE)
# but your template is in
# print(router_template)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [ ]:
from langchain.chains.router import MultiPromptChain

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )


chain.run("How do magnets work?")
chain.run("How do Feynman Diagrams work?")                        

# Transform chain
- save some money by transforming yourself instead of asking llm to transform
- uses a seq chain to chain a transform func and an instruction

In [ ]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
yelp_review ='''
TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!
REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!
FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.
NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!
THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!
THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KNOWLEDGEABLE ABOUT THE MENU. THEY WENT ABOVE AND BEYOND TO ENSURE THAT WE HAD THE BEST DINING EXPERIENCE POSSIBLE.
AND LET'S NOT FORGET ABOUT THE DESSERTS! OH MY, OH MY! I HAD THEIR SIGNATURE CHOCOLATE LAVA CAKE, AND IT WAS PURE HEAVEN. THE CAKE WAS MOIST, AND WHEN I CUT INTO IT, THE WARM CHOCOLATE OOOZED OUT, CREATING AN EXPLOSION OF FLAVOR IN MY MOUTH. IT WAS LIKE A SYMPHONY OF CHOCOLATEY GOODNESS!
IN CONCLUSION, THIS RESTAURANT IS A HIDDEN GEM! IF YOU WANT TO INDULGE IN A MEMORABLE DINING EXPERIENCE, DO YOURSELF A FAVOR AND VISIT THIS PLACE. YOU WON'T REGRET IT! I CAN'T WAIT TO GO BACK AND TRY MORE OF THEIR DELECTABLE DISHES. KUDOS TO THE ENTIRE TEAM FOR CREATING SUCH A CULINARY HAVEN!
ALL I CAN SAY IS... WOOHOO!
'''

print(yelp_review)

In [ ]:
#(inputs: dict) -> dict = dict input dict output
def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want, 
    just make sure to reference it correct in the chain call.
    '''
    # GRAB INCOMING CHAIN TEXT
    text = inputs['text']

    #grab review section only
    only_review_text = text.split('REVIEW:')[-1]
    
    #lower case it
    lower_case_text = only_review_text.lower()
    
    #return a dict
    return {'output':lower_case_text}

In [ ]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun)


template = "Create a one sentence summary of this review:\n{review_text}"

In [ ]:
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(template)
summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="review_summary")


sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain],
                                        verbose=True)


result = sequential_chain(yelp_review)

print(result['output'])
print(result['input'])